In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from functions import *

In [37]:
device = torch.device('cpu')
word_dict = load_pickle(   '../data/word_dict.pkl' )
seq_train = load_pkl2ndarr( '../data/input/train_proteins.pkl' )
target_train = load_pkl2ndarr( '../data/input/train_normtopts.pkl' )
datapack = [seq_train, target_train]
batch_data = [datapack[di][8: 12] for di in range(len(datapack))]
words, target_values = data2tensor(batch_data, True, device)

In [38]:
# embeddding-attention-cnn1d-dense layer

In [39]:
#embedding
dim=64
window=5
embed_word = nn.Embedding(len(word_dict)+1, dim)
x = embed_word(words)
x = x.transpose(1,2)
print(x.size())

torch.Size([4, 64, 570])


In [40]:
values_conv = nn.Conv1d(dim, dim, kernel_size=2*window+1, padding=window)
weights_conv = nn.Conv1d(dim, dim, kernel_size=2*window+1, padding=window)

In [45]:
values = values_conv(x)
weights = weights_conv(x)
weights = F.softmax(weights, dim=-1)
xa = values * weights
print(weights.size())
print(xa.size())

torch.Size([4, 64, 570])
torch.Size([4, 64, 570])


In [51]:
xa_mean = torch.mean(xa, dim=-1)
xa_max,_ = torch.max(xa,dim=-1)
pf = torch.cat([xa_mean, xa_max], dim=1)

In [52]:
pf.size()

torch.Size([4, 128])

In [1]:
import torch
import esm
import pandas as pd
import numpy as np
from functions import *
from model import PredOT

In [2]:
train_data = pd.read_csv('../data/train_os.csv')
T_max, T_min = 120, 0
train_pack = [np.array(train_data.uniprot_id), np.array(train_data.sequence), \
                  np.array( rescale_targets(list(train_data.topt), T_max, T_min)) ];
batch_data = [train_pack[di][3:8] for di in range(len(train_pack))]
ids, seqs, topts = batch_data
device = torch.device('cpu')
M = PredOT( device, 5, 0.5, 3, 4)
M = M.half() #float16
M.to(device);

In [6]:
pred = M( ids, seqs )

RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'

In [2]:
data = [
    ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
    ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
    ("protein3","KALTARQQEVFDLIRDISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEERILERSKEPVSGAQLAEELSVSRQVIVQDI") ]

In [3]:
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval();

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt" to /home/bras5181/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t6_8M_UR50D-contact-regression.pt" to /home/bras5181/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D-contact-regression.pt


In [4]:
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

In [5]:
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[6], return_contacts=True)

In [6]:
results["representations"][6].size()

torch.Size([3, 102, 320])